# Setting up the benchmark
In order to compare different methods with possibly different parameters, we need to set up a few things before running the benchmark. A `Benchmark` object receives some input parameters to configure the test:
- A task: This could be `denoising` or `detecting`. The first one compare the quality reconstruction factor at the output of the method, whereas the second simply consist in detecting wheter a signal is present or not.
- A dictionary of methods: Each entry of this dictionary should correspond to the function that implements each of the desired methods.
- A dictionary of parameters: Each entry of this dictionary correspond to an array of tuples to pass as input parameters to each method. In order to know which parameters should be passed to each methods, the keys of this dictionary should be those corresponding to the individual methods in the corresponding dictionary.
- SNR vector: A list or tuple of values of SNR to test.
- A number of repetitions.

## A dummy test 
First let's define a dummy method for testing. Methods should receive an `MxN` array of noisy signals, where `M` is the number of signals, and `N` is the number of time samples Additionally, it should receive a second parameter `params` to test different combinations of parameters of the method. The shape of the output depends on the task (denoising or detecting). After this, let's create a dictionary of methods to pass the `Benchmark` object later.

In [3]:
import numpy as np
from numpy import pi as pi
import pandas as pd
from benchmark import Benchmark


def a_method(noisy_signals, params):
    # If additional input parameters are needed, they can be passed in a tuple using params and then parsed.
    results = noisy_signals 
    return results

# Create a dictionary of the methods to test.
my_methods = {
    'Method 1': a_method, 
    'Method 2': a_method
    }

Create a dictionary of the parameters to pass to each method. The parameters should be given in a tuple of tuples, so that each internal tuple is passed as a single additional parameter to the method (this latter should implement how to parse the parameters from this tuple). The keys of this dictionary should be the same as those of the methods dictionary.

In [4]:
my_parameters = {
    'Method 1': [[3, 4, True, 'all'],[2, 1, False, 'one']], # Each tuple is passed as a parameter that the method should parse internally.
    'Method 2': [[3, 4, True, 'all'],[2, 1, False, 'one']]
    }

Now we are ready to instantiate a `Benchmark` object, run a test using the proposed methods and parameters. The benchmark constructor receives a name of a task (which defines the performance function for the test), a dictionary of the methods to test, the desired length of the signals used in the simulation, a dictionary of different parameters that should be passed to the methods, an array with different values of SNR to test, and the number of repetitions that should be used for each test.

In [5]:
my_benchmark = Benchmark(task = 'denoising', methods = my_methods, N = 256, parameters = my_parameters, SNRin = [40,50], repetitions = 3)
my_results = my_benchmark.runTest() # Run the test. my_results is a dictionary with the results for each of the variables of the simulation.

Running benchmark...
The test has finished.


Now we have the results of the test in `my_results`. In order to get the results in a human-readable way using a DataFrame, and also for further analysis and reproducibility, we could use the method `getResults()`.

In [6]:
df = my_benchmark.getResults() # This formats the results on a DataFrame
df.head()

,Method,Parameter,Signal_id,Repetition,40,50
6,Method 1,Params0,cosChirp,0,40.000000,50.000000
7,Method 1,Params0,cosChirp,1,40.000000,50.000000
8,Method 1,Params0,cosChirp,2,40.000000,50.000000
12,Method 1,Params0,crossedLinearChirps,0,39.999997,49.999967
13,Method 1,Params0,crossedLinearChirps,1,39.999997,49.999967


## Passing different parameters to the methods.
It is common that a method depend on certain parameters (thresholds, multiplicative factors, etc). We can pass a number of parameters to our method provided it parses them internally. For instance, let's consider a function that depends on two thresholds:

In [7]:
def a_function(signal, thr1, thr2):
    return signal[signal>thr1]

Now lets create the method and the dictionary for our benchmark. Notice that the method function should parse the parameters in `params`. Then we can define the diferent combinations of parameters to test using another dictionary:

In [8]:
def another_method(noisy_signals,params):
    output_signals = np.zeros(noisy_signals.shape)
    # The method should parse the parameters accordingly
    thr1 = params[0]
    thr2 = params[1]
    output = [a_function(signal, thr1, thr2) for signal in noisy_signals]
    return output
        

# Create a dictionary of the methods to test.
my_methods = {
    'Method 1': another_method, 
    }


# Create a dictionary for the different values of the thresholds to test:
thr1 = np.linspace(0.1,0.5,0.1)
thr2 = np.linspace(4,6,0.5)
my_parameters = {
    'Method 1': 
}

SyntaxError: invalid syntax (3820171951.py, line 21)